# Семинар 1: Эмбеддинги слов

Сегодня мы поиграем с эмбеддингами слов: обучим наши собственные маленькие эмбеддинги, загрузим один из них из gensim и используем его для визуализации текстовых корпусов.

Если хотите запустить локально, то не забудьте

```bash
pip install --upgrade nltk gensim bokeh
```

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt

--2024-02-13 10:50:31--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2024-02-13 10:50:31--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd6c748b229f27f9219091cdfe.dl.dropboxusercontent.com/cd/0/get/CNOKMiURB00DDJG-fH727cbzxZXSu8pA1_cRWJ_voTpWfBwNbIDD_16unZc_UE2_SFTSN80MsXO-VM4K0hoTRLqQltVm1WVQR3H8G_oo1iJG6VEZsYktxH033b5G18n0yxo/file?dl=1# [following]
--2024-02-13 10:50:32--  https://uccd6c748b229f27f9219091cdfe.dl.dropboxusercontent.com/cd/0/get/CNOKMiURB00DDJG-fH727cbzxZXSu8pA1_cRWJ_voTpWfBwNbIDD_16unZc_UE2_SFTSN80MsXO-VM4K0hoTRLqQltVm1WVQR3H8G_oo1iJG6VEZsYktxH033b5G18n0yx

In [ ]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

__Токенизация:__ стандартный первый шаг в NLP пайплайне.
Текст, с которым мы работаем, представлен сырой: со всеми знаками препинания, смайликами и так далее, поэтому простой str.split не подойдет.

Используем __`nltk`__ - библиотеку для решения многих задач NLP, таких как токенизация, стемминг или определение частей речи.

In [ ]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

NameError: name 'data' is not defined

In [ ]:
# TASK: все в нижнем регистре и извлекаем лексемы с помощью токенизатора.
# data_tok должен быть списком со списков токенов для каждой строки в данных.

data_tok = # YOUR CODE

In [ ]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "преобразуйте каждую строку в список токенов (строк)."
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "преобразуйте каждую строку в список токенов (строк)."
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "пожалуйста, убедитесь, что данные написаны в нижнем регистре"

In [ ]:
print([' '.join(row) for row in data_tok[:2]])

__Word vectors:__ кроме упомянутых на лекции способов моделей есть *GloVe* с разными целевыми функциями и *fasttext*, который использует модели на уровне символов для обучения вкраплений слов. Все методы довольно похожи, fasttext позволяет оперировать неизвестными словами, разбивая их на части.

Выбор огромен, поэтому начнем с малого: __gensim__ - еще одна библиотека NLP, в которой представлено множество векторных моделей, в том числе и word2vec.

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [ ]:
# now you can get word vectors !
model.get_vector('anything')

In [ ]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

### Претренированные модели

Как вы заметили, процесс не быстрый. А теперь представьте себе обучение эмбеддингов слов на гигабайтах текста: статьях в Википедии или сообщениях в X.

К счастью, существует куча предобученных моделей.

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [ ]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

### Визуализация векторов

Один из способов проверить, насколько хороши наши векторы - отобразить их на графике.

Заодно вспомним о  методах уменьшения _размерности_.

Построим scatterplot 1000 наиболее часто встречающихся слов.

In [ ]:
words = model.index_to_key[:1000]

print(words[::100])

In [ ]:
# for each word, compute it's vector with model
word_vectors = # YOUR CODE

In [ ]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### PCA


In [ ]:
from sklearn.decomposition import PCA

# Отобразите векторы слов на двумерную плоскость с помощью PCA. Используйте старый добрый sklearn api (fit, transform)
# после этого нормализуйте векторы, чтобы убедиться, что у них нулевое среднее и единичная дисперсия
word_vectors_pca = # YOUR CODE

# and maybe MORE OF YOUR CODE here :)

In [ ]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "для каждого слова должен существовать 2d-вектор"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "точки должны быть центрированы"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "точки должны иметь единичную дисперсию"

#### Рисуем!

В этот раз интерактивные графики с библиотекой bokeh.

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# наведите на точки курсор мыши и посмотрите, сможете ли вы назвать как-то кластеры

### t-SNE
PCA хорош, но он строго линеен и, следовательно, способен отразить только грубую высокоуровневую структуру данных.

Если забыли, то вот небольшой материал для повторения: __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [ ]:
from sklearn.manifold import TSNE

# отобразите векторы слов на двумерную плоскость с помощью TSNE. подсказка: не паникуйте, на подгонку может уйти минута или две.
# нормализуйте их так же, как и в pca


word_tsne = #YOUR CODE

In [ ]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

### Визуализация фраз

Эмбеддинги слов также можно использовать для представления коротких фраз. Самый простой способ - взять __среднее__ векторов для всех лексем во фразе с некоторыми весами.

Этот трюк полезен для того, чтобы определить, с какими данными вы работаете: найти, есть ли какие-либо выбросы, кластеры или какие-то артефакты.


In [ ]:
def get_phrase_embedding(phrase):
    """
    Преобразуйте фразу в вектор, объединив в нем вкрапления слов. См. описание выше.
    """
    # 1. переведите фразу в нижний регистре
    # 2. токенизируйте фразу
    # 3. усредните векторы для всех слов в токенизированной фразе
    # пропускаем слова, которых нет в словарe модели
    # если все слова отсутствуют в словарe, возвращаются нули

    vector = np.zeros([model.vector_size], dtype='float32')

    # YOUR CODE

    return vector



In [ ]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [ ]:
# рассмотрим только ~5k фраз
chosen_phrases = data[::len(data) // 1000]

# рассчитайте векторы фраз
phrase_vectors = # YOUR CODE

In [ ]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [1]:
# отобразите векторы в двумерное пространство с помощью pca, tsne или другого выбранного вами метода
# не забудьте нормализовать

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

NameError: name 'TSNE' is not defined

In [ ]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [ ]:
# вычислите векторы для всех строк
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [ ]:
def find_nearest(query, k=10):
    """
    вернуть k наиболее похожих строк из данных, отсортированных от наиболее до наименее похожих
    сходство должно измеряться как косинус между векторами запроса и строки
    можно использовать глобальные переменные: data и data_vectors. см. также: np.argpartition, np.argsort
    """
    # YOUR CODE

    return <YOUR CODE: top-k lines starting from most similar>

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Что еще?__
* Посмотрите, что еще есть в model zoo: `gensim.downloader.info()`
* Попробуйте [FastText](https://github.com/facebookresearch/fastText)